

# Runnables: Definition and Characteristics  
A Runnable is a fundamental unit in LangChain that provides a standard interface for various components with the following key capabilities:

### Core Characteristics of Runnables  
A unit of work that can be:  
- **Invoked** (transform a single input into an output)  
- **Batched** (transform multiple inputs efficiently)  
- **Streamed** (produce outputs incrementally)  
- **Composed** (combined with other Runnables)  

### Composition Primitives  
LangChain provides two main composition primitives for creating Runnables:  

#### 1. RunnableSequence  
- Allows chaining multiple Runnables sequentially  
- The output of one Runnable becomes the input for the next  
- Can be created using the pipe operator (`|`) or explicitly with `RunnableSequence`  

**Example:**  
```python
chain = runnable1 | runnable2  # Sequential execution
# Output of runnable1 feeds into runnable2
```

#### 2. RunnableParallel  
- Runs multiple Runnables concurrently with the same input  
- Returns a dictionary with results from each Runnable  
- Executes Runnables in parallel, significantly reducing processing time  

**Example:**  
```python
parallel_chain = RunnableParallel({
    "key1": runnable1,
    "key2": runnable2
})
# Both runnables run concurrently with the same input
```

### Additional Composition Features  
- Automatic type coercion (functions become `RunnableLambda`, dictionaries become `RunnableParallel`)  
- Support for async and sync execution  
- Optimized parallel processing using thread pools  

### Benefits of LCEL Composition  
- Optimized parallel execution  
- Guaranteed async support  
- Simplified streaming  
- Seamless tracing with LangSmith  
- Standard API across different components  


In [ ]:
#from dotenv import load_dotenv

#load_dotenv('../env')

In [ ]:
from langchain_ollama import ChatOllama

base_url = "http://localhost:11434"
model = 'llama3.2:1b'
llm = ChatOllama(
    base_url=base_url,
    model=model,
    temperature=0.5,
    num_predict=512
)

# Runnable Interface Demo

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["topic", "number"],
    template="Please explain {topic} using exactly {number} detailed and distinct use cases."
)

chain = (
    prompt_template 
    | llm
)
type(chain)


In [ ]:
chain

In [ ]:
# Invoke the chain with input values
response = chain.invoke({
    "topic": "langchain", 
    "number": 2
})

# Print the response content
print(response.content)


### Additional Examples of Runnable Interface Methods

#### 1. Batch Processing
The `batch_transaction_analysis` function demonstrates how to process multiple transactions simultaneously using the `batch` method of the `transaction_chain`. This is particularly useful for analyzing large datasets efficiently.

#### 2. Single Invoke
The `single_transaction_invoke` function showcases the `invoke` method, which processes a single transaction input. This is ideal for scenarios where individual transaction analysis is required.

#### 3. Streaming Transaction Processing
The `stream_transaction_processing` function illustrates the `stream` method, which processes transactions incrementally and outputs results in real-time. This is beneficial for handling high-value or time-sensitive transactions.


## Create a prompt template for transaction analysis

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Analyze the following payment transaction and provide insights: {transaction}"
)

## Define a function to process transaction details

In [ ]:
def process_transaction_details(transaction):
    """
    Simulate additional transaction processing logic
    In a real-world scenario, this might involve:
    - Fraud detection checks
    - Compliance verification
    - Additional metadata enrichment
    """
    return f"Processed transaction: {transaction}"

## Construct the processing chain
The processing chain is constructed by combining multiple Runnables sequentially. The `transaction_chain` is built using the following components:

1. **Prompt Template (`prompt`)**: This defines the structure of the input prompt for analyzing transactions. It ensures that the input transaction details are formatted appropriately for the LLM.

2. **LLM (`llm`)**: The language model processes the formatted prompt and generates insights or analysis based on the transaction details.

3. **Processing Function (`process_transaction_details`)**: This function applies additional logic to the LLM's output, such as simulating fraud detection, compliance checks, or metadata enrichment.

The chain is created using the pipe operator (`|`), which sequentially connects these components. The output of one component becomes the input for the next, enabling streamlined and modular processing of transaction data.


In [ ]:
transaction_chain = (
    prompt |  # Apply the prompt template
    llm |  # Generate analysis using LLM
    process_transaction_details  # Additional processing
)

In [ ]:
transaction_chain

## Demonstration of different Runnable Interface methods

### 1. Batch Processing
The `batch_transaction_analysis` function demonstrates how to process multiple transactions simultaneously using the `batch` method of the `transaction_chain`. This method is particularly useful for analyzing large datasets efficiently. The function takes a list of transactions, processes them in a batch, and prints the analysis results for each transaction.


In [ ]:
def batch_transaction_analysis():
    print("--- Batch Processing Transactions ---")
    transactions = [
        "Credit card payment of $500 from customer ABC",
        "International wire transfer of $10,000",
        "Recurring subscription payment of $49.99"
    ]
    
    # Batch process multiple transactions
    batch_results = transaction_chain.batch(transactions)
    
    for i, result in enumerate(batch_results, 1):
        print(f"Transaction {i} Analysis: {result}")

### 2. Single Invoke
The `single_transaction_invoke` function demonstrates the `invoke` method of the `transaction_chain`. This method processes a single transaction input and generates an analysis result. It is particularly useful for scenarios where individual transaction analysis is required. The function takes a single transaction as input, invokes the chain, and prints the resulting analysis.


In [ ]:
def single_transaction_invoke():
    print("\n--- Single Transaction Invoke ---")
    transaction = "Refund processing for order #12345"
    
    # Invoke the chain for a single transaction
    result = transaction_chain.invoke(transaction)
    print("Transaction Analysis:", result)

### 3. Streaming Transaction Processing
The `stream_transaction_processing` function demonstrates the `stream` method of the `transaction_chain`. This method processes a transaction incrementally and outputs results in real-time. It is particularly useful for handling high-value or time-sensitive transactions. The function takes a single transaction as input, streams the analysis results chunk by chunk, and prints them as they are generated.

In [ ]:
def stream_transaction_processing():
    print("\n--- Streaming Transaction Processing ---")
    transaction = "High-value international payment of $250,000"
    
    print("Streaming analysis for transaction:")
    for chunk in transaction_chain.stream(transaction):
        print(chunk, end="", flush=True)
    print()  # New line after streaming

In [ ]:

if __name__ == "__main__":
    print("=== Batch Transaction Analysis ===")
    batch_transaction_analysis()

    print("\n=== Single Transaction Invoke ===")
    single_transaction_invoke()

    print("\n=== Streaming Transaction Processing ===")
    stream_transaction_processing()